In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [5]:
data = data.loc[data['Year']== '2015']
data

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
13836,15962,From Out Here,The Advisory Circle,jon brooks’ compositions as the advisory circl...,Ghost Box,7.0,January,13,2015,Electronic,NaN,NaN,NaN
13839,15965,Individ,The Dodos,progress can be a hard thing to quantify. for ...,Polyvinyl,7.2,January,19,2015,Rock,NaN,NaN,NaN
13841,15967,Cowboy Worship EP,Amen Dunes,damon mcmahon has spent much of his life both ...,Sacred Bones,6.9,January,19,2015,Rock,NaN,NaN,NaN
13842,15968,Irreal,Disappears,disappears left plenty of space in the cracks ...,Kranky,7.0,January,19,2015,Rock,NaN,NaN,NaN
13843,15969,Euclid,Kaitlyn Aurelia Smith,"there’s a gentle kind of alchemy at work on ,,...",Western Vinyl,7.3,January,19,2015,Experimental,NaN,NaN,NaN
13844,15970,Teaspoon to the Ocean,Jib Kidder,if you’ve ever stayed up to an ungodly hour se...,Weird World,6.3,February,4,2015,Rock,NaN,NaN,NaN
13845,15971,Signs Under Test,John Tejada,with a career that’s almost passed the two-dec...,Kompakt,7.4,February,4,2015,Electronic,NaN,NaN,NaN
13846,15972,Flyaway Garden,Breakfast in Fur,"the college town of new paltz, new york is sit...",Bar/None,7.6,February,4,2015,Rock,NaN,NaN,NaN
13847,15973,Reflection,Fifth Harmony,"women reclaimed the pop charts in 2014, sparki...","Epic,Syco",7.2,February,11,2015,Pop/R&B,NaN,NaN,NaN
13848,15974,Home,Phantom Posse,as the de facto leader of the collective phant...,Self-released,7.3,February,4,2015,Pop/R&B,NaN,NaN,NaN


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['content']))

print(data_words[:1])

[['jon', 'brooks', 'compositions', 'as', 'the', 'advisory', 'circle', 'immaculately', 'turn', 'around', 'one', 'another', 'irrespective', 'of', 'the', 'varying', 'moods', 'that', 'beset', 'him', 'during', 'recording', 'he', 'moves', 'from', 'the', 'outdoorsy', 'pine', 'of', 'his', 'record', 'to', 'the', 'more', 'inward', 'facing', 'carefully', 'trailing', 'links', 'to', 'make', 'it', 'sound', 'recognizably', 'him', 'it', 'there', 'in', 'the', 'perfectly', 'interlaced', 'synth', 'lines', 'the', 'air', 'of', 'cold', 'war', 'nostalgia', 'the', 'overriding', 'sense', 'that', 'utopian', 'thought', 'has', 'become', 'past', 'construct', 'brooks', 'works', 'quietly', 'releasing', 'music', 'under', 'varying', 'guises', 'never', 'looking', 'for', 'attention', 'it', 'fits', 'with', 'his', 'fondness', 'for', 'barely', 'heralded', 'musicians', 'synth', 'pioneers', 'who', 'never', 'got', 'their', 'dues', 'whose', 'work', 'now', 'fuels', 'small', 'part', 'of', 'the', 'reissues', 'industry', 'this', '

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['jon', 'brooks', 'compositions', 'as', 'the', 'advisory', 'circle', 'immaculately', 'turn', 'around', 'one', 'another', 'irrespective', 'of', 'the', 'varying', 'moods', 'that', 'beset', 'him', 'during', 'recording', 'he', 'moves', 'from', 'the', 'outdoorsy', 'pine', 'of', 'his', 'record', 'to', 'the', 'more', 'inward', 'facing', 'carefully', 'trailing', 'links', 'to', 'make', 'it', 'sound', 'recognizably', 'him', 'it', 'there', 'in', 'the', 'perfectly', 'interlaced', 'synth', 'lines', 'the', 'air', 'of', 'cold', 'war', 'nostalgia', 'the', 'overriding', 'sense', 'that', 'utopian', 'thought', 'has', 'become', 'past', 'construct', 'brooks', 'works', 'quietly', 'releasing', 'music', 'under', 'varying', 'guises', 'never', 'looking', 'for', 'attention', 'it', 'fits', 'with', 'his', 'fondness', 'for', 'barely', 'heralded', 'musicians', 'synth', 'pioneers', 'who', 'never', 'got', 'their', 'dues', 'whose', 'work', 'now', 'fuels', 'small', 'part', 'of', 'the', 'reissues', 'industry', 'this', 'i

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['brook', 'composition', 'advisory', 'circle', 'immaculately', 'turn', 'irrespective', 'vary', 'mood', 'beset', 'record', 'move', 'outdoorsy', 'pine', 'record', 'inward', 'face', 'carefully', 'trail', 'link', 'make', 'sound', 'recognizably', 'perfectly', 'interlaced', 'synth', 'line', 'air', 'cold', 'war', 'nostalgia', 'override', 'sense', 'utopian', 'thought', 'become', 'construct', 'brook', 'work', 'quietly', 'release', 'music', 'vary', 'guise', 'never', 'look', 'attention', 'fit', 'fondness', 'barely', 'herald', 'musician', 'synth', 'pioneer', 'never', 'get', 'due', 'whose', 'work', 'fuel', 'small', 'part', 'reissue', 'industry', 'music', 'longing', 'lose', 'much', 'find', 'make', 'knowledge', 'process', 'fall', 'view', 'end', 'coloring', 'work', 'unique', 'texture', 'brook', 'find', 'broad', 'sound', 'palette', 'explore', 'even', 'work', 'name', 'good', 'track', 'often', 'non', 'musical', 'component', 'tick', 'clock', 'keep', 'rhythm', 'upwardly', 'twirl', 'synth', 'pattern', 'win

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 4), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 2), (109, 1), (110, 1)

In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.074*"jame" + 0.043*"excursion" + 0.000*"entertainment" + '
  '0.000*"cover_art" + 0.000*"rhode" + 0.000*"bump" + 0.000*"disappoint" + '
  '0.000*"rework" + 0.000*"bonus" + 0.000*"unite"'),
 (1,
  '0.005*"folk" + 0.005*"arrangement" + 0.005*"cover" + 0.005*"blue" + '
  '0.005*"string" + 0.004*"offer" + 0.004*"acoustic" + 0.003*"studio" + '
  '0.003*"simple" + 0.003*"dark"'),
 (2,
  '0.010*"jazz" + 0.009*"musician" + 0.008*"film" + 0.008*"instrument" + '
  '0.008*"electronic" + 0.006*"material" + 0.006*"rhythm" + 0.006*"composer" + '
  '0.006*"recording" + 0.005*"stretch"'),
 (3,
  '0.100*"faith" + 0.050*"pursuit" + 0.048*"pitchfork" + 0.039*"precious" + '
  '0.037*"virtue" + 0.033*"expert" + 0.029*"sneer" + 0.024*"association" + '
  '0.003*"authority" + 0.000*"east"'),
 (4,
  '0.164*"rap" + 0.109*"rapper" + 0.041*"mixtape" + 0.035*"tape" + '
  '0.026*"flow" + 0.023*"verse" + 0.019*"trap" + 0.019*"nigga" + '
  '0.017*"street" + 0.017*"rapping"'),
 (5,
  '0.014*"punk" + 0.007*"m

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.917909601703012

Coherence Score:  0.4077906420166513


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.054930  0.008341       1        1  36.491234
1      0.115563 -0.030744       2        1  22.006819
2      0.114995  0.039305       3        1  13.673676
5      0.065655 -0.021366       4        1  12.162067
9      0.096166  0.117003       5        1  11.703306
7     -0.222821 -0.310906       6        1   2.313617
4     -0.325633  0.242805       7        1   1.471091
3      0.021678 -0.015499       8        1   0.132065
0      0.038289 -0.014712       9        1   0.039574
6      0.041178 -0.014228      10        1   0.006553, topic_info=     Category        Freq          Term       Total  loglift  logprob
668   Default  426.000000           rap  426.000000  30.0000  30.0000
1078  Default  284.000000        rapper  284.000000  29.0000  29.0000
1024  Default  229.000000         verse  229.000000  28.0000  28.0000
425   Default  209.000000          tape  209.000000  27.0000  27.0000
930   Default  199.000000          girl  199.000000  26.0000  26.0000
989   Default  313.000000        sample  313.000000  25.0000  25.0000
439   Default  321.000000       version  321.000000  24.0000  24.0000
711   Default  311.000000         dance  311.000000  23.0000  23.0000
336   Default  387.000000            ep  387.000000  22.0000  22.0000
1147  Default  293.000000          punk  293.000000  21.0000  21.0000
50    Default  179.000000        future  179.000000  20.0000  20.0000
1031  Default  186.000000         woman  186.000000  19.0000  19.0000
1294  Default  151.000000           bar  151.000000  18.0000  18.0000
367   Default  246.000000          jazz  246.000000  17.0000  17.0000
1035  Default  293.000000         young  293.000000  16.0000  16.0000
1201  Default  282.000000         house  282.000000  15.0000  15.0000
77    Default  254.000000      musician  254.000000  14.0000  14.0000
2042  Default  207.000000          free  207.000000  13.0000  13.0000
2477  Default  123.000000          flow  123.000000  12.0000  12.0000
332   Default  309.000000    electronic  309.000000  11.0000  11.0000
2567  Default  106.000000       mixtape  106.000000  10.0000  10.0000
1161  Default  131.000000        street  131.000000   9.0000   9.0000
47    Default  200.000000          film  200.000000   8.0000   8.0000
475   Default  232.000000    instrument  232.000000   7.0000   7.0000
2100  Default  109.000000          disc  109.000000   6.0000   6.0000
382   Default  196.000000      original  196.000000   5.0000   5.0000
100   Default  114.000000       reissue  114.000000   4.0000   4.0000
2267  Default  103.000000         money  103.000000   3.0000   3.0000
418   Default  261.000000          star  261.000000   2.0000   2.0000
387   Default  169.000000   performance  169.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
3429  Topic10    0.003372     cultivate   26.956045   0.6465  -8.1403
1151  Topic10    0.003372        scream   92.550926  -0.5870  -8.1403
1149  Topic10    0.003372       revival   22.175259   0.8418  -8.1403
1148  Topic10    0.003372        regard   35.933708   0.3591  -8.1403
1147  Topic10    0.003372          punk  293.391144  -1.7408  -8.1403
1146  Topic10    0.003372        propel   12.976459   1.3776  -8.1403
1145  Topic10    0.003372       primary   45.288551   0.1277  -8.1403
1144  Topic10    0.003372  presentation   21.964306   0.8513  -8.1403
1143  Topic10    0.003372       popular   87.829178  -0.5346  -8.1403
1142  Topic10    0.003372          pick  139.591248  -0.9980  -8.1403
1141  Topic10    0.003372          peer  120.385719  -0.8500  -8.1403
1140  Topic10    0.003372          pack   65.531906  -0.2418  -8.1403
1139  Topic10    0.003372           owe   34.696346   0.3941  -8.1403
1138  Topic10    0.003372         night  189.063324  -1.3013  -8.1403
1137  Topic10    0.003372  neighborhood   25.288727   0.7104  -8.1403
1136  Topic10    0.

In [21]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print(topic_idx)
        print.join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print (documents[doc_index])